# Dataland Pipeline – Demo Notebook

Dieses Notebook ruft die CLI auf und zeigt die erzeugten Dateien in `data/raw/`.

```bash
!pip install -e ..
!dataland run --company "Siemens" --log-level INFO
```


## MVP-Pipeline: Dataland → Transform → Mock-Rooms → AI & Visualisierung

In diesem Notebook bauen wir Schritt für Schritt eine Daten-Pipeline, die:
1. **Nachhaltigkeitsdaten** von Unternehmen aus der Dataland-API holt
2. Diese Daten **transformiert** und in ein einheitliches Format bringt
3. Die Daten in **Mock-Rooms** (lokale Dateien) speichert
4. **KI-basierte Analysen** und **Visualisierungen** ermöglicht

---

### Was ist Dataland?

Dataland ist eine Plattform, die ESG-Daten (Environmental, Social, Governance) von Unternehmen sammelt und über eine API bereitstellt. ESG-Daten umfassen z.B.:
- CO₂-Emissionen
- Energieverbrauch
- Mitarbeiter-Kennzahlen
- Governance-Strukturen

### Was ist ein Connector?

Ein Connector ist ein Programmteil, der:
- Eine **Verbindung** zu einer externen Datenquelle aufbaut
- Die Daten **abruft** (meist über HTTP-Requests)
- Die Rohdaten für die weitere Verarbeitung **bereitstellt**

---

## 1. Setup & Konfiguration

Zuerst installieren und importieren wir alle benötigten Python-Bibliotheken. → TODO ggf. ändern zu "Im Verlauf des Projekts werden mehrere Bibliotheken installiert und importiert, um verschiedene Aufgaben wie Datenabruf, -verarbeitung und Dateiverwaltung zu ermöglichen."

### Was machen diese Bibliotheken?

- **requests**: Ermöglicht HTTP-Anfragen an APIs (wie Dataland)
- **python-dotenv**: Lädt Variablen aus der .env-Datei (z. B. den API-Token)
- **pydantic**: Verwaltet Konfigurationswerte über ein Validierungs-Modell (Settings in config.py)
- **typer**: Baut die Kommandozeilen-Oberfläche (dataland run ...) auf
- **logging** (Standardbibliothek):	Für strukturierte Logausgaben in der Konsole
- **json** (Standardbibliothek): Verarbeitet JSON-Daten (das Standard-Datenformat von APIs)
- **hashlib** (Standardbibliothek):	Erzeugt eindeutige Hashes zur Duplikaterkennung bei Datenpunkten
- **pathlib** (Standardbibliothek): Moderner Weg, um mit Dateipfaden zu arbeiten
- **datetime / timezone** (Standardbibliothek): Hilft beim Arbeiten mit Datum und Zeit - Zeitstempel für Logs und Persistenz
- **time** (Standardbibliothek): Zum Messen von Laufzeiten und Backoff-Pausen
- **typing** (Standardbibliothek): Typannotationen für bessere Lesbarkeit und Autovervollständigung

- **os**: Ermöglicht Zugriff auf das Dateisystem → TODO ggf. nicht mehr enthalten?

---

## 2. Konfiguration der Dataland-API

Bevor wir Daten abrufen können, müssen wir festlegen:
- Wo ist die API? (Base URL)
- Wie authentifizieren wir uns? (API Key)
- Welches Unternehmen interessiert uns?

### Wichtig: API-Zugang

In diesem MVP verwenden wir zunächst **Mock-Daten** oder öffentlich zugängliche Endpoints. Für den Produktivbetrieb würden Sie:
1. Einen Account bei Dataland erstellen
2. Einen API-Key erhalten
3. Diesen sicher in einer Umgebungsvariable speichern

---

## 3. Helper-Funktionen für die API-Kommunikation

Bevor wir Daten abrufen, erstellen wir wiederverwendbare Funktionen.

### Was macht ein guter Connector?

Ein professioneller Connector sollte:
- **Fehler behandeln**: Was passiert, wenn die API nicht antwortet?
- **Authentifizierung**: Korrektes Senden von API-Keys
- **Logging**: Was wird gerade gemacht? Gab es Probleme?
- **Retry-Logik**: Bei temporären Fehlern automatisch nochmal versuchen
- **Pagination**: Große Datenmengen in Teilen abrufen

---

## 3. JSONL Raw-Persistenz Funktionen

Bevor wir Daten von der API abrufen, erstellen wir Funktionen zum Speichern der Rohdaten.

### Was ist Raw-Persistenz?

**Raw-Persistenz** bedeutet, dass wir die Daten **exakt so** speichern, wie wir sie von der API bekommen:
- ✅ Keine Transformation
- ✅ Keine Normalisierung  
- ✅ Komplette API-Response
- ✅ Metadaten zum Request (Zeitstempel, Endpoint, Status)

### Warum JSONL?

**JSONL** (JSON Lines) ist perfekt für unseren Use-Case:
- Eine Zeile = ein API-Response
- Einfach zu erweitern (append-only)
- Später leicht zu verarbeiten
- Fehlertoleranz (eine kaputte Zeile ≠ kaputte Datei)

### Envelope-Format

Jede Zeile enthält ein "Envelope" mit Metadaten:

```json
{
  "ts": "2025-11-05T13:45:12Z",
  "endpoint": "/api/metadata/companies/search",
  "status": 200,
  "request": {"params": {"q": "Siemens"}},
  "payload": { /* Original API-Response */ }
}
```

---

## 4. HTTP-Session mit Retry-Logik & Authentifizierung

Jetzt erstellen wir eine robuste HTTP-Session für die API-Kommunikation.

### Was macht diese Session besonders?

1. **Authentifizierung**: Bearer-Token automatisch in jedem Request
2. **Retry-Logik**: Bei temporären Fehlern automatisch wiederholen
3. **Exponential Backoff**: Bei Rate Limits (HTTP 429) intelligente Wartezeiten
4. **Fehlerbehandlung**: Unterscheidung zwischen permanenten und temporären Fehlern
5. **Logging**: Detaillierte Ausgaben für Debugging

### Welche Fehler werden behandelt?

- **HTTP 429** (Too Many Requests): Rate Limit erreicht → Exponential Backoff
- **HTTP 5xx** (Server-Fehler): Temporärer Fehler → Retry
- **Timeout**: Netzwerk-Problem → Retry
- **HTTP 4xx** (außer 429): Client-Fehler → Kein Retry (z.B. 404, 401)

---

## 🔍 SCHRITT 1a: COMPANY-LOOKUP VIA API SERVICE (Primärweg)

**Ziel:** Finde die `company_id` über den API-Service (nicht Documents).

**Problem:** Die Swagger-Dokumentation zeigt keinen eindeutigen `/metadata/companies/search` Endpoint. Daher implementieren wir einen **systematischen Probing-Ansatz**:

### Strategie:

1. **Probe-Liste:** Teste verschiedene Endpoint-Kandidaten aus der Konfiguration
2. **Persistenz:** Speichere alle Versuche in `raw/company_lookup_api.jsonl`
3. **Fallback:** Wenn API-Lookup fehlschlägt → Documents-Service nutzen

### Warum API-Service bevorzugen?

- Direkte `company_id` ohne Umweg über Documents
- Konsistent mit `/metadata/available-data-dimensions` Flow
- Bessere Performance (keine Document-Metadaten laden)

### Typische Response:

```json
{
  "companies": [
    {
      "companyId": "dataland_siemens_ag_001",
      "name": "Siemens AG",
      "sector": "Industrials",
      "country": "DE"
    }
  ]
}
```

---

---

## 📊 SCHRITT 2: VERFÜGBARE DIMENSIONEN ABRUFEN

**Jetzt, wo wir eine `company_id` haben,** fragen wir die API: **Welche Daten sind für dieses Unternehmen verfügbar?**

### Was sind "Dimensions"?

**Available Data Dimensions** (`/metadata/available-data-dimensions`) ist ein Metadaten-Endpoint, der zurückgibt:
- Welche **Datenpunkte** (Indicators) verfügbar sind
- Für welche **Perioden** (Jahre, Quartale) Daten vorliegen
- Welche **Dimensionen** (z.B. Scope 1/2/3 bei CO₂) existieren
- **IDs**, die wir für den Datenabruf brauchen

### Swagger-Dokumentation:

Dieser Endpoint ist offiziell dokumentiert:
- **URL:** `https://dataland.com/api/swagger-ui/index.html`
- **Ressource:** `/metadata/available-data-dimensions`
- **Parameter:** `companyId` (Pflicht)

### Beispiel-Response:

```json
{
  "availableDataPoints": [
    {
      "dataPointId": "co2_scope1_2023",
      "period": "2023",
      "dimension": "Scope1",
      "indicator": "CO2_Emissions",
      "unit": "Mt"
    },
    {
      "dataPointId": "energy_total_2023",
      "period": "2023",
      "indicator": "Energy_Consumption",
      "unit": "TWh"
    }
  ]
}
```

### Warum wichtig?

Ohne Dimensions wissen wir nicht, **welche Daten überhaupt abrufbar sind**. Jede Dimension liefert eine `dataPointId`, die wir im nächsten Schritt für den Value-Abruf brauchen.

---

---

## 7. Values ziehen: Alle Daten systematisch abrufen

Jetzt kommt der Hauptteil: Für **jeden** Datenpunkt aus den Dimensions rufen wir die tatsächlichen **Werte** ab.

### Strategie: "Alles Verfügbare"

Wir iterieren über alle Dimensions-Einträge und rufen für jeden:
- Den **Wert** (Value)
- Die **Metadaten** (Einheit, Quelle, Qualität)
- Weitere **Kontexte** (Scope, Kategorie, etc.)

### Idempotenz & Duplicate-Vermeidung

Um zu verhindern, dass wir denselben Datenpunkt mehrfach ziehen:
- Generieren wir einen **Hash** aus `(company_id, data_point_id, period)`
- Prüfen, ob dieser Hash schon in einer Set-Variable existiert
- Speichern nur, wenn neu

### Fehlertoleranz

- Einzelne Fehler (z.B. 404 für einen Datenpunkt) stoppen nicht die gesamte Pipeline
- Jeder Fehler wird geloggt
- Am Ende: Zusammenfassung von Erfolgen und Fehlern

### Progress-Tracking

Bei vielen Datenpunkten (>100):
- Zeigen wir alle 10% den Fortschritt an
- Geben Zwischenstatistiken aus

---

## 8. (Optional) Dokumente & Sustainability Reports

Zusätzlich zu den strukturierten Daten (Metriken) können wir auch **Dokumente** abrufen:
- Sustainability Reports (Nachhaltigkeitsberichte)
- Annual Reports (Geschäftsberichte)
- Weitere PDF/Text-Dokumente

### Warum Dokumente?

Dokumente enthalten:
- ✅ **Textuelle Kontexte** für AI/Q&A
- ✅ **Strategien & Narrative** (nicht nur Zahlen)
- ✅ **Qualitative Informationen**
- ✅ **Erklärungen** zu den Metriken

### Ablauf:

1. **Suche** nach Dokumenten für das Unternehmen
2. **Metadaten** abrufen (Titel, Jahr, Typ, URL)
3. **Speichern** in `raw/document_search.jsonl`

### Hinweis:

Der vollständige Download der PDF-Dateien ist **optional** und nicht Teil dieses MVPs. Wir speichern zunächst nur die Metadaten und URLs.

---

## 9. Run-Log & Gesamt-Statistiken

Zum Abschluss erstellen wir ein **Run-Log**, das alle wichtigen Informationen über diesen Durchlauf speichert.

### Was enthält das Run-Log?

- ✅ **Zeitstempel** (Start, Ende, Dauer)
- ✅ **Konfiguration** (Company Query, Base URL)
- ✅ **Statistiken** (Erfolge, Fehler, Records)
- ✅ **HTTP-Session-Stats** (Requests, Retries, Rate Limits)
- ✅ **Fehler-Zusammenfassung**
- ✅ **Compliance-Info** (z.B. API-Nutzungsbedingungen)

### Warum ist das wichtig?

Das Run-Log ermöglicht:
- 📊 **Monitoring**: Wie gut läuft die Pipeline?
- 🐛 **Debugging**: Was lief schief?
- 📝 **Audit**: Wann wurden welche Daten gezogen?
- ⚖️ **Compliance**: Nachweis der regelkonformen Nutzung

---

## 10. HTTP-Session Statistiken & Abschluss

Zum Abschluss zeigen wir alle HTTP-Statistiken und eine finale Zusammenfassung.

### Was haben wir erreicht? ✅

1. **Company-Resolve**: Aus "Siemens" wurde eine eindeutige `company_id`
2. **Available Dimensions**: Liste aller verfügbaren Datenpunkte geholt
3. **Values**: Systematisch alle Werte abgerufen
4. **Documents** (optional): Dokument-Metadaten gesammelt
5. **Raw-Persistenz**: Alles in JSONL gespeichert
6. **Run-Log**: Vollständige Dokumentation des Durchlaufs

### Nächste Schritte

Im nächsten Teil der Pipeline:
- **Transform Layer**: Rohdaten ins kanonische Schema überführen
- **Mock-Rooms**: Events strukturiert speichern
- **AI & Visualisierung**: Use-Cases implementieren

---

## 🎭 MOCK-DATEN (Entwicklungs-Fallback)

Falls **weder API noch Documents** erfolgreich waren, erstellen wir Mock-Daten für die Entwicklung.

### Wann wird Mock-Modus aktiviert?

- Kein `DATALAND_TOKEN` in `.env`
- Alle Company-Lookup Endpoints fehlgeschlagen (HTTP 404/401)
- Network-Probleme (Timeout, Firewall)
- Dataland-API offline

### Was wird gemockt?

1. **Company Info:** Mock-Unternehmen mit fester `company_id`
2. **Dimensions:** 6 typische ESG-Dimensionen (CO₂, Energie, Wasser, etc.)
3. **Values:** Realistische Werte für Testdaten
4. **Documents:** 2 Mock-Sustainability-Reports

### Vorteil:

Du kannst die **gesamte Pipeline testen**, ohne Zugang zur echten API. Ideal für:
- Entwicklung ohne API-Zugang
- Offline-Arbeit
- CI/CD Tests
- Demo-Präsentationen

---

---

## 4. Connector initialisieren und testen

Jetzt erstellen wir eine Instanz unseres Connectors und testen die Verbindung.

---

## 5. Mock-Daten für Entwicklung und Tests

**Wichtiger Hinweis:** Da wir möglicherweise noch keinen echten Dataland-API-Zugang haben, erstellen wir Mock-Daten, die realistisch sind.

### Warum Mock-Daten?

Mock-Daten erlauben es uns:
- **Ohne API-Zugang** zu entwickeln
- **Reproduzierbare Tests** durchzuführen
- Die **Pipeline-Logik** zu testen, ohne auf externe Systeme angewiesen zu sein
- **Kosten zu sparen** (viele APIs sind kostenpflichtig)

Später können wir die Mock-Daten einfach durch echte API-Calls ersetzen.

---

## 6. Rohdaten inspizieren

Schauen wir uns die Struktur der abgerufenen Daten genauer an. Das ist wichtig, um zu verstehen:
- Welche Felder vorhanden sind
- Wie die Daten strukturiert sind
- Was wir im Transform-Schritt verarbeiten müssen

---

## 7. Datenqualität prüfen

Bevor wir die Daten weiterverarbeiten, sollten wir einige grundlegende Qualitätsprüfungen durchführen.

### Was prüfen wir?

- Sind alle erwarteten Felder vorhanden?
- Sind die Datentypen korrekt?
- Gibt es fehlende Werte?
- Sind die Werte plausibel?

---

## 8. Zusammenfassung: Connector/Ingest

### Was haben wir erreicht? ✅

1. **Connector-Klasse erstellt**: Eine professionelle, wiederverwendbare Klasse für API-Kommunikation
2. **Fehlerbehandlung**: Retry-Logik, Timeouts, HTTP-Error-Handling
3. **Mock-Daten**: Realistische Testdaten für die Entwicklung
4. **Validierung**: Qualitätsprüfung der eingehenden Daten

### Datenübersicht

Wir haben folgende Rohdaten gesammelt:

---

## ✅ Company-Lookup: Robuste Fallback-Strategie implementiert!

### 🎯 Implementierte Lösung

Da die Dataland-Swagger **keinen eindeutigen Company-Search-Endpoint** dokumentiert, haben wir eine **dreistufige Fallback-Strategie** implementiert:

#### 1️⃣ **Primärweg: API-Service Probing**
```python
search_company_via_api(api_session, query, raw_dir)
```

**Probiert systematisch:**
- `/metadata/companies/search?q=Siemens`
- `/metadata/companies?query=Siemens`
- `/companies/search?q=Siemens`
- `/companies?name=Siemens`
- `/entities/search?q=Siemens`
- `/api/companies/search?q=Siemens`

**Persistenz:** Alle Versuche → `raw/company_lookup_api.jsonl`

#### 2️⃣ **Fallback: Documents-Service Probing**
```python
search_company_via_documents(doc_session, query, raw_dir)
```

**Probiert systematisch:**
- `/?q=Siemens`
- `/search?q=Siemens`
- `/?query=Siemens`
- `/search?query=Siemens`
- `/companies/search?q=Siemens`
- `/documents/search?q=Siemens`

**Persistenz:** Alle Versuche → `raw/company_lookup_documents.jsonl`

#### 3️⃣ **Last-Resort: Mock-Daten**
```python
create_mock_data_if_needed()
```

Falls beide Wege fehlschlagen, aktiviert sich automatisch der Mock-Modus.

---

### 📊 Vorteile dieser Architektur

| Feature | Beschreibung |
|---------|--------------|
| **Resilience** | System funktioniert auch bei API-Änderungen |
| **Debugging** | Alle Probes in JSONL → einfache Fehleranalyse |
| **Flexibility** | Neue Endpoints via CONFIG hinzufügen |
| **Transparency** | Klare Logs zeigen, welcher Weg erfolgreich war |
| **Offline-Ready** | Mock-Modus für Entwicklung ohne API |

---

### 🔍 Swagger-Recherche Ergebnis

**API-Swagger:** `https://dataland.com/api/swagger-ui/index.html`
- ✅ `/metadata/available-data-dimensions` (dokumentiert)
- ❌ `/metadata/companies/search` (nicht gefunden)
- ❓ Company-Lookup-Endpoint unklar

**Documents-Swagger:** `https://dataland.com/documents/swagger-ui/index.html`
- ✅ Document-Operationen (exists, update, extend)
- ❌ Expliziter Company-Search (nicht gefunden)

**→ Daher: Probing-Ansatz ist die robusteste Lösung!**

---

### 🧪 Nächste Schritte

1. **Teste das Notebook:**
   ```bash
   # Erstelle .env mit Token
   echo "DATALAND_TOKEN=your_token_here" > .env
   
   # Führe Notebook aus
   jupyter notebook mvp_pipeline.ipynb
   ```

2. **Analysiere Logs:**
   ```bash
   # Welcher Endpoint war erfolgreich?
   cat raw/company_lookup_api.jsonl | jq '.status'
   cat raw/company_lookup_documents.jsonl | jq '.status'
   ```

3. **Update CONFIG:**
   - Falls ein Endpoint erfolgreich ist → nur diesen in CONFIG behalten
   - Reduziere Probing-Liste auf funktionierende Endpoints

4. **Transform-Layer:**
   - JSONL → MetricEvents & TextEvents
   - Mock-Rooms Schema
   - Visualisierungen

---

### 💡 Für Produktiv-Umgebung

Sobald du den **exakten funktionierenden Endpoint** identifiziert hast:

```python
# Anstatt 6 Probes:
CONFIG["company_lookup_api_candidates"] = [
    {"path": "/metadata/companies/search", "params": {"q": None}}  # ← nur dieser
]
```

Das reduziert API-Calls und verbessert Performance!

---

---

## 🏗️ Architektur-Übersicht: Company-Lookup mit Fallback

```mermaid
graph TD
    A[Start: Company Query 'Siemens'] --> B{API-Service Probing}
    
    B -->|6 Endpoint-Kandidaten| C[/metadata/companies/search]
    B --> D[/metadata/companies]
    B --> E[/companies/search]
    B --> F[/companies]
    B --> G[/entities/search]
    B --> H[/api/companies/search]
    
    C -->|HTTP 200| SUCCESS1[✅ Company ID gefunden]
    D -->|HTTP 200| SUCCESS1
    E -->|HTTP 200| SUCCESS1
    F -->|HTTP 200| SUCCESS1
    G -->|HTTP 200| SUCCESS1
    H -->|HTTP 200| SUCCESS1
    
    C -->|HTTP 404/401| FALLBACK{Documents-Service Probing}
    D -->|HTTP 404/401| FALLBACK
    E -->|HTTP 404/401| FALLBACK
    F -->|HTTP 404/401| FALLBACK
    G -->|HTTP 404/401| FALLBACK
    H -->|HTTP 404/401| FALLBACK
    
    FALLBACK -->|6 Endpoint-Kandidaten| I[/]
    FALLBACK --> J[/search]
    FALLBACK --> K[/companies/search]
    
    I -->|HTTP 200| SUCCESS2[✅ Company ID gefunden]
    J -->|HTTP 200| SUCCESS2
    K -->|HTTP 200| SUCCESS2
    
    I -->|HTTP 404/401| MOCK[🎭 Mock-Modus aktivieren]
    J -->|HTTP 404/401| MOCK
    K -->|HTTP 404/401| MOCK
    
    SUCCESS1 --> NEXT[Weiter zu Dimensions]
    SUCCESS2 --> NEXT
    MOCK --> NEXT
    
    NEXT --> DIMS[/metadata/available-data-dimensions]
    DIMS --> VALUES[/data-points/values]
    VALUES --> DOCS[/documents/search]
    DOCS --> END[✅ Ingest Complete]
    
    style SUCCESS1 fill:#90EE90
    style SUCCESS2 fill:#90EE90
    style MOCK fill:#FFD700
    style END fill:#90EE90
```

**Legende:**
- 🟢 **Grün:** Erfolgreicher Pfad (Company ID gefunden)
- 🟡 **Gelb:** Fallback-Modus (Mock-Daten)
- **Alle Probes:** Werden in JSONL persistiert für Debugging

---